In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cugraph

In [2]:
df = pd.read_csv("../../Graph_Data_Storage/homo.csv")
src=df.loc[:,"source"].to_numpy()
dst=df.loc[:,"target"].to_numpy()

In [3]:
df_info = pd.read_csv("../../Graph_Data_Storage/homo_info.csv")
no_nodes = df_info.loc[:,"No. Nodes"].to_numpy()
no_nodes = no_nodes[0]
print(no_nodes)
adj_mat = np.zeros(shape=(no_nodes,no_nodes))

2000


In [4]:
for s,d in zip(src,dst):
    adj_mat[s][d]=1
    
adj_T = adj_mat.T

In [5]:
g = nx.from_numpy_array(adj_mat, create_using=nx.DiGraph())


In [6]:
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(g)
nx.draw_networkx(g, pos=pos, arrows=True)  # Set arrows=True to make the graph directed
labels = {i: i for i in range(len(adj_mat))}
nx.draw_networkx_labels(g, pos=pos)
plt.show()


KeyboardInterrupt: 

<Figure size 800x600 with 0 Axes>

In [6]:
init_guess = np.random.rand(np.shape(adj_mat)[0], 1)
init_guess = init_guess / np.sum(init_guess)

In [7]:
## Generate CSC format in order to randomly pluck edges
from scipy.sparse import csc_matrix, csr_matrix
from scipy.sparse.csgraph import connected_components

nonzero_indices = np.nonzero(adj_T)
nonzero_tuples = list(zip(nonzero_indices[0], nonzero_indices[1]))
print(nonzero_tuples)
num_nonzero_edges = len(nonzero_tuples)
print(num_nonzero_edges)



[(50, 0), (50, 1), (50, 2), (50, 3), (50, 4), (50, 5), (50, 6), (50, 7), (50, 8), (50, 9), (50, 10), (50, 11), (50, 12), (50, 13), (50, 14), (50, 15), (50, 16), (50, 17), (50, 18), (50, 19), (50, 20), (50, 21), (50, 22), (50, 23), (50, 24), (50, 25), (50, 26), (50, 27), (50, 28), (50, 29), (50, 30), (50, 31), (50, 32), (50, 33), (50, 34), (50, 35), (50, 36), (50, 37), (50, 38), (50, 39), (50, 40), (50, 41), (50, 42), (50, 43), (50, 44), (50, 45), (50, 46), (50, 47), (50, 48), (50, 49), (51, 0), (51, 1), (51, 2), (51, 3), (51, 4), (51, 5), (51, 6), (51, 7), (51, 8), (51, 9), (51, 10), (51, 11), (51, 12), (51, 13), (51, 14), (51, 15), (51, 16), (51, 17), (51, 18), (51, 19), (51, 20), (51, 21), (51, 22), (51, 23), (51, 25), (51, 26), (51, 27), (51, 28), (51, 29), (51, 30), (51, 31), (51, 32), (51, 33), (51, 34), (51, 35), (51, 36), (51, 37), (51, 38), (51, 39), (51, 40), (51, 41), (51, 42), (51, 43), (51, 44), (51, 45), (51, 46), (51, 47), (51, 48), (51, 49), (51, 50), (52, 0), (52, 1), (

In [8]:
##Generate the six different adjacency matrices for a "ring of computation"

adj_1 = adj_T.copy()
adj_2 = adj_T.copy()
adj_3 = adj_T.copy()
adj_4 = adj_T.copy()
adj_5 = adj_T.copy()
adj_6 = adj_T.copy()
init_guess_1 = init_guess.copy()
init_guess_2 = init_guess.copy()
init_guess_3 = init_guess.copy()
init_guess_4 = init_guess.copy()
init_guess_5 = init_guess.copy()
init_guess_6 = init_guess.copy()
list_edges_1 = np.random.rand(num_nonzero_edges)
list_edges_2 = np.random.rand(num_nonzero_edges)
list_edges_3 = np.random.rand(num_nonzero_edges)
list_edges_4 = np.random.rand(num_nonzero_edges)
list_edges_5 = np.random.rand(num_nonzero_edges)
list_edges_6 = np.random.rand(num_nonzero_edges)

lin_threshold = .25
iterations = 10

In [9]:
for i in range(iterations):
    for i in range(len(list_edges_1)):
        if list_edges_1[i] > lin_threshold:
            row,col = nonzero_tuples[i][0], nonzero_tuples[i][1]
            adj_1[row][col] = 0
    init_guess_1 = adj_1@init_guess_1
    ## Adjacency matrix 2
    for i in range(len(list_edges_2)):
        if list_edges_2[i] > lin_threshold:
            row,col = nonzero_tuples[i][0], nonzero_tuples[i][1]
            adj_2[row][col] = 0
    init_guess_2 = adj_2@init_guess_2
    ## Adjacency matrix 3
    for i in range(len(list_edges_3)):
        if list_edges_3[i] > lin_threshold:
            row,col = nonzero_tuples[i][0], nonzero_tuples[i][1]
            adj_3[row][col] = 0
    init_guess_3 = adj_3@init_guess_3
    ## Adjacency matrix 4

    for i in range(len(list_edges_4)):
        if list_edges_4[i] > lin_threshold:
            row,col = nonzero_tuples[i][0], nonzero_tuples[i][1]
            adj_4[row][col] = 0
    init_guess_4 = adj_4@init_guess_4
    ## Adjacency matrix 5
    for i in range(len(list_edges_5)):
        if list_edges_5[i] > lin_threshold:
            row,col = nonzero_tuples[i][0], nonzero_tuples[i][1]
            adj_5[row][col] = 0
    init_guess_5 = adj_5@init_guess_5
    ## Adjacency matrix 6
    for i in range(len(list_edges_6)):
        if list_edges_6[i] > lin_threshold:
            row,col = nonzero_tuples[i][0], nonzero_tuples[i][1]
            adj_6[row][col] = 0
    init_guess_6 = adj_6@init_guess_6

    list_edges_1 = np.random.rand(num_nonzero_edges)
    list_edges_2 = np.random.rand(num_nonzero_edges)
    list_edges_3 = np.random.rand(num_nonzero_edges)
    list_edges_4 = np.random.rand(num_nonzero_edges)
    list_edges_5 = np.random.rand(num_nonzero_edges)
    list_edges_6 = np.random.rand(num_nonzero_edges)

In [10]:
print(init_guess_1)
print(init_guess_2)
print(init_guess_3)
print(init_guess_4)
print(init_guess_5)
print(init_guess_6)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [12]:
 # Replace with your vector
all_zero = all(element == 0 for element in init_guess_1)
print(all_zero)
all_zero = all(element == 0 for element in init_guess_2)
print(all_zero)
all_zero = all(element == 0 for element in init_guess_3)
print(all_zero)
all_zero = all(element == 0 for element in init_guess_4)
print(all_zero)
all_zero = all(element == 0 for element in init_guess_5)
print(all_zero)
all_zero = all(element == 0 for element in init_guess_6)
print(all_zero)


True
True
True
True
True
True
